# Experiment and path specification

In [ ]:
# Specify the experiment for analysis
# experiment='cxic00318'
# runNumber = 123
# experiment='cxix40218'
experiment='cxilu9218'
# runNumber=31 # includes cspad info
# runNumber=32 # includes cspad info
runNumber=33 # includes cspad info
runNumber=29 # includes cspad info
runNumber=28 # includes cspad info
runNumber=27 # includes cspad info
runNumber=26 # includes cspad info
runNumber=14 # includes cspad info
runNumber=13 # includes cspad info
runNumber=21 # includes cspad info
runNumber=37 # includes cspad info
runNumber=39 # includes cspad info
runNumber=49 # includes cspad info
runNumber=50 # includes cspad info
runNumber=14 # includes cspad info
runNumber=49 # includes cspad info
runNumber=73 # includes cspad info
# runNumber=21 # includes cspad info
# runNumber=9 # includes cspad info


# Set the installation and output path
import os
os.environ['INSTALLPATH']= '/cds/home/m/mrware/TRXS-dev/TRXS-Run18v3'
os.environ['OUTPUTPATH']= '/cds/data/psdm/%s/%s/scratch' % (experiment[0:3],experiment)

In [ ]:
# x,y = CSPADgeometry(detType='Jungfrau', run=runNumber, experiment=experiment)

In [ ]:
# np.savez('detgeom',x=x,y=y)

In [ ]:
detGeom = np.load('/cds/home/m/mrware/TRXS-dev/TRXS-Run18v3/Template-Notebooks/detgeom.npz')
x = detGeom['x']
y = detGeom['y']
r = np.sqrt(y**2+x**2)
rMax = 1e5
NR = 20
dr = float(rMax)/float(NR)
idx = 20
np.sum((r>idx*dr)&(r< (idx+1)*dr ) )

# Import Libraries

In [ ]:
# Magic iPython command to enable plotting
%matplotlib inline

# Load in the pythonBatchMagic library
import sys
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/pythonBatchMagic')
from pythonBatchMagic import *

# Determine current user
currentUser, error = unixCMD("echo $USER")
currentUser = currentUser.strip()
print(currentUser)
os.environ['RESULTSPATH']= ('/cds/data/psdm/%s/%s/results/%s' % (experiment[0:3],experiment,currentUser)).strip()
if not os.path.exists(os.environ['RESULTSPATH']):
    os.mkdir(os.environ['RESULTSPATH'])

# Leveraging the batch queue to quickly grab point data

## Loading libraries

In [ ]:
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/LCLS')
from LCLSdefault import *

sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/mattsLibraries')
from picklez import *

# Load in the get data library
from dataAnalysis import *

# Load in the batch library for lcls
from lclsBatch import *

## Grabbing point data across nodes

In [ ]:
detDict = {
    'uvint':{'name': 'Acqiris','get-function':'getAcqirisSum2'},
           'uvint1':{'name': 'Acqiris','get-function':'getAcqirisSum1'},
           'gasPressure':{'name': 'CXI:MKS670:READINGGET','get-function':'getGasPressure'},
           'xrayEnergy':{'name':'FEEGasDetEnergy','get-function': 'getXrayEnergy' },
           'ebeamcharge':{'name':'EBeam', 'get-function': 'getEbeamCharge'},
          'stageencoder':{'name':'CXI:LAS:MMN:04.RBV','get-function':'getStageEncoder'},
          'ttfltpos':{'name':'CXI:TIMETOOL:FLTPOS', 'get-function': 'getTTFltPos'},
          'vitpos':{'name':'LAS:FS5:VIT:FS_TGT_TIME_DIAL', 'get-function': 'getTTFltPos'},
           'xint':{'name':'CxiDg3_Imb01','get-function': 'getIPM' },
           'xpos':{'name':'CxiDg3_Imb01','get-function': 'getXPos' },
           'ypos':{'name':'CxiDg3_Imb01','get-function': 'getYPos' },
          'seconds':{'name':'seconds', 'get-function': 'getSeconds'},
          'nanoseconds':{'name':'nanonseconds', 'get-function': 'getNanoseconds'},
          'fiducials':{'name':'fiducials', 'get-function': 'getFiducials'},
#           'roi1':{'name':'jungfrau4M', 'get-function': 'getCSPADsum1'},
#           'roi2':{'name':'jungfrau4M', 'get-function': 'getCSPADsum2'},
#           'roi3':{'name':'jungfrau4M', 'get-function': 'getCSPADsum3'},
#           'roi4':{'name':'jungfrau4M', 'get-function': 'getCSPADsum4'},
#           'roi4':{'name':'jungfrau4M', 'get-function': 'getCSPADsum4'},
#           'rois':{'name':'jungfrau4M', 'get-function': 'getCSPADrois'},
          'rois':{'name':'jungfrau4M', 'get-function': 'getRadialrois'},
          'laserOn':{'name':'evr1', 'get-function': 'getLaserOn'},
          'xrayOn':{'name':'evr1', 'get-function': 'getXrayOn'},
          'photonHist':{'name':'jungfrau4M', 'get-function': 'getPhotonHistogram'}
          }



NSHOTS = 60000
ngrab = 200
rank = int( float(NSHOTS)/float(ngrab) )
print('Grabbing %d shots, %d events at a time, on %d nodes' %(NSHOTS,ngrab,rank))
batchThreads = batchPointDataGrabber( detDict, eventMax=int(rank*ngrab), experiment=experiment, runNumber=runNumber, rank=int(rank) )
# batchThreads.Queue = 'psanaq'
batchThreads.Queue = 'psfehhiprioq'
batchThreads.RunType = 'python2'
batchThreads.start()

## Check job status

In [ ]:
stdoutdata, stderrdata = bjobs()
print stdoutdata
print stderrdata


## Kill threads if they die

In [ ]:
batchThreads.requestStop()
# time.sleep(5)
stdoutdata, stderrdata = bkill(killAll=True)
print stdoutdata
print stderrdata

In [ ]:
batchThreads.status

In [ ]:
batchThreads.gather()

## Drop empty array values

In [ ]:
batchThreads.detArrays['rois'].shape

In [ ]:
pointData = batchThreads.detArrays
# print pointData

def dropEmpty(pointData):

    pdc = {}
    filledBin = (pointData['seconds'] > 0)

    for key in pointData.keys():
        try:
            pdc[key]= np.copy(pointData[key][filledBin,:])
        except IndexError as ie:
            pdc[key]= np.copy(pointData[key][filledBin] )
        
    return pdc

def dropNaN(pointData):
    goodBin = np.ones(pointData['seconds'].shape, dtype=bool)
    for key in pointData.keys():
        try:
            NX, NY = pointData[key].shape
            print(pointData[key].shape)
            goodBin = goodBin & ~np.isnan( np.sum(pointData[key],-1 ) )
        except Exception:
            goodBin = goodBin & ~np.isnan( pointData[key].astype(float) )
            
    for key in pointData.keys():
        try:
            pointData[key]=pointData[key][goodBin,:] 
        except IndexError as ie:
            pointData[key]=pointData[key][goodBin] 
    return pointData

print len(pointData['seconds'])
pointData_masked = dropNaN(dropEmpty(pointData))
print 'Number of good shots: ', len(pointData_masked['seconds'])
print( np.sum( (pointData['seconds'] > 0) ) )

## Save point data

In [ ]:
save_obj( pointData, os.environ['RESULTSPATH']+'/pointData-run-%d' % runNumber )

In [ ]:
print(os.environ['RESULTSPATH']+'/pointData-run-%d' % runNumber)

In [ ]:
# pointData = load_obj(os.environ['RESULTSPATH']+'/pointData-run-%d' % 38)

## Plot histogram of result

In [ ]:
detArrays = pointData

fig2, ax2 = plt.subplots(1, 1)
ax2.hist(detArrays['stageencoder'], bins=100,normed=False, alpha=0.5);
print(detArrays['xrayEnergy'].shape)
# plt.savefig('runfolder/hist-xrayenergy')

In [ ]:
adu = np.arange(200) * 10
plt.semilogy(adu, np.nansum(detArrays['photonHist'],0));
plt.xlabel('adu')
plt.ylabel('counts')
# plt.xlim(250,500)

In [ ]:
detArrays = pointData

fig2, ax2 = plt.subplots(1, 1)
ax2.hist(detArrays['gasPressure'], bins=100,normed=False, alpha=0.5);
# print(detArrays['xrayEnergy'].shape)
# plt.savefig('runfolder/hist-xrayenergy')

In [ ]:
detArrays['rois']

In [ ]:
detArrays = pointData

fig2, ax2 = plt.subplots(1, 1)
ax2.hist(detArrays['rois'][:,15], bins=100,normed=False, alpha=0.5);
# print(detArrays['xrayEnergy'].shape)
# plt.savefig('runfolder/hist-xrayenergy')

In [ ]:
x = np.linspace(0,1.5e5, 20)
plt.plot(x,np.nanmean(detArrays['rois'][1:],0))
plt.xlabel('pixel pos')
plt.ylabel('mean adu')


In [ ]:
plt.scatter(detArrays['rois'][detArrays['xrayOn'].astype(bool),3],detArrays['rois'][detArrays['xrayOn'].astype(bool),1])

In [ ]:
detArrays = pointData

fig2, ax2 = plt.subplots(1, 1)
ax2.hist(detArrays['xrayOn'], bins=100,normed=False, alpha=0.5);

In [ ]:
detArrays = pointData

fig2, ax2 = plt.subplots(1, 1)
ax2.hist(detArrays['laserOn'], bins=100,normed=False, alpha=0.5);

# Get labtime to check for systematics

In [ ]:
import datetime
import matplotlib.dates as dates
def getLabtime(second, microseconds):
    refdate = datetime.datetime(1970,1,1)
    td = datetime.timedelta(seconds=second,microseconds=microseconds)
    return refdate+td

labtime = np.array([getLabtime( s, float(ns)/1000. ) for s,ns in zip(detArrays['seconds'],detArrays['nanoseconds'])])

In [ ]:
fig,ax= plt.subplots()
date_form = dates.DateFormatter('%H:%M', tz=None)
ax.plot(labtime, detArrays['roi1'], )
ax.xaxis.set_major_formatter(date_form)

# Check for correlation between rois

In [ ]:
plt.scatter(detArrays['roi2'],detArrays['roi1'])
plt.xlabel('Jungfrau sum 2')
plt.ylabel('Jungfrau sum 1')

# Look at ROIs vs. stage position

In [ ]:
goodIdx = detArrays['laserOn'] & detArrays['xrayOn'] & ( detArrays['rois'][:,10] > 0 )
pos = detArrays['stageencoder'][goodIdx] 
np.unique(pos)

In [ ]:
nb=5
bins = np.unique(pos)

counts,edges = np.histogram( pos,bins=np.unique(pos))
bins,edges = np.histogram( pos,bins=np.unique(pos), weights=(roi2)/(roi1+roi2+roi3))

In [ ]:
plt.plot(edges[1:] , bins/counts )
plt.xlabel(' CXI:LAS:MMN:04.RBV ')
plt.ylabel(' roi1/roi2 ')
# plt.savefig('run-13-t0.png')